## Proyecto: **Sistema Seguro de Gestión de Cuentas y Tareas**

**Framework principal:** FastAPI

**GUI para configuración:** Streamlit

**Autenticación:** JWT

**Base de datos:** PostgreSQL 

**Logs:** Loguru

**Documentación:** Swagger (FastAPI)

**Protección por roles:** `admin` y `user`



---


## Estructura general por capas

```
secure_task_manager/
├── app/
│   ├── main.py                  # Entry point FastAPI
│   ├── core/                    # Configuración y utilidades
│   │   ├── config.py            # Carga del .env
│   │   ├── logger.py            # Logger con Loguru
│   ├── models/                  # Modelos SQLAlchemy
│   │   └── user_task.py
│   ├── schemas/                 # Esquemas Pydantic
│   │   └── user_task.py
│   ├── services/                # Lógica de negocio
│   │   ├── auth_service.py
│   │   └── task_service.py
│   ├── routes/                  # Rutas y controladores
│   │   ├── auth_routes.py
│   │   └── task_routes.py
│   ├── database/                # DB engine y Base
│   │   └── session.py
├── streamlit_app.py             # GUI para configurar la clave
├── .env                         # Clave secreta
├── requirements.txt
└── README.md
```

---


Requisitos (requirements.txt)

In [ ]:
fastapi
uvicorn
sqlalchemy
pydantic
passlib[bcrypt]
python-jose[cryptography]
python-dotenv
streamlit
psycopg2-binary



core/config.py

In [ ]:
# Importa el módulo 'os' que permite interactuar con variables de entorno del sistema operativo
import os

# Importa la función 'load_dotenv' que carga las variables de un archivo .env al entorno del sistema
from dotenv import load_dotenv

# Carga las variables definidas en el archivo .env al entorno de ejecución
load_dotenv()

# Obtiene la variable de entorno 'SECRET_KEY' del archivo .env.
# Si no existe, se usará "clave_secreta_super_segura" por defecto.
SECRET_KEY = os.getenv("SECRET_KEY", "clave_secreta_super_segura")

# Define el algoritmo de firma que usará el token JWT (usualmente HS256)
ALGORITHM = "HS256"

# Define el tiempo de expiración en minutos del token JWT
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# Obtiene las credenciales de conexión a PostgreSQL desde las variables de entorno
POSTGRES_USER = os.getenv("POSTGRES_USER")         # Nombre de usuario de la base de datos
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD") # Contraseña del usuario
POSTGRES_DB = os.getenv("POSTGRES_DB")             # Nombre de la base de datos
POSTGRES_HOST = os.getenv("POSTGRES_HOST")         # Dirección del host (ej: localhost o IP)
POSTGRES_PORT = os.getenv("POSTGRES_PORT")         # Puerto de conexión (por defecto: 5432)

# Construye la cadena de conexión a PostgreSQL usando los valores anteriores
DATABASE_URL = (
    f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}"  # Usuario y contraseña
    f"@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"    # Host, puerto y base de datos
)


core/logger.py

In [ ]:
# Importa el módulo estándar de logging para registrar mensajes en archivos o consola
import logging

# Importa el módulo os para interactuar con el sistema de archivos (crear carpetas, etc.)
import os

# Importa un handler especial que permite rotar los logs automáticamente en intervalos de tiempo (por ejemplo, diario)
from logging.handlers import TimedRotatingFileHandler

# Función que inicializa y configura el sistema de logging
def init_logger():
    # Crea la carpeta "logs" si no existe, para guardar allí los archivos de log
    os.makedirs('logs', exist_ok=True)

    # Obtiene el logger raíz (global)
    logger = logging.getLogger()

    # Establece el nivel mínimo de mensajes que se van a registrar (INFO o superior: INFO, WARNING, ERROR, CRITICAL)
    logger.setLevel(logging.INFO)

    # Crea un handler que guarda los logs en "logs/app.log", y los rota cada medianoche
    # - backupCount=7: guarda hasta 7 archivos de log antiguos (uno por día)
    # - encoding='utf-8': garantiza que el texto se guarde correctamente en UTF-8
    handler = TimedRotatingFileHandler(
        'logs/app.log', when='midnight', backupCount=7, encoding='utf-8'
    )

    # Define el formato del mensaje que se va a registrar en cada línea del log
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    # Asigna el formato definido al handler
    handler.setFormatter(formatter)

    # Agrega el handler al logger solo si aún no tiene uno (evita agregar múltiples veces si se llama más de una vez)
    if not logger.handlers:
        logger.addHandler(handler)


database/session.py

In [ ]:
# Importa el constructor del motor de base de datos de SQLAlchemy
from sqlalchemy import create_engine

# Importa herramientas para manejar sesiones y clases base de modelos
from sqlalchemy.orm import sessionmaker, declarative_base

# Importa la URL de conexión a la base de datos definida en el archivo de configuración
from core.config import DATABASE_URL

# Crea un motor de conexión a la base de datos usando la URL configurada (PostgreSQL en este caso)
engine = create_engine(DATABASE_URL)

# Crea una clase fábrica para generar sesiones de conexión a la base de datos.
# - autoflush=False: no se escriben automáticamente los cambios hasta que se haga commit
# - autocommit=False: se requiere llamar manualmente a commit() para guardar los cambios
SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False)

# Crea una clase base para que los modelos ORM de SQLAlchemy la usen como superclase
Base = declarative_base()

# Función generadora que se utiliza como dependencia en FastAPI
# Abre una sesión de base de datos, y la cierra al finalizar
def get_db():
    db = SessionLocal()   # Se crea una nueva sesión
    try:
        yield db          # Se retorna la sesión para usarla en los endpoints
    finally:
        db.close()        # Al finalizar, se asegura que la conexión se cierre correctamente


models/user_task.py

In [ ]:
# Importa las clases necesarias para definir columnas y tipos de datos en la tabla
from sqlalchemy import Column, Integer, String, Boolean

# Importa la clase base que permite a SQLAlchemy registrar este modelo como una tabla
from database.session import Base

# Define la clase User como un modelo de SQLAlchemy que representa la tabla 'users'
class User(Base):
    # Nombre de la tabla que se creará en la base de datos
    __tablename__ = "users"

    # Columna 'id': clave primaria, tipo entero, autoincremental por defecto
    id = Column(Integer, primary_key=True)

    # Columna 'username': tipo texto, debe ser único en la tabla
    username = Column(String, unique=True)

    # Columna 'hashed_password': tipo texto, donde se guarda la contraseña ya cifrada (hash)
    hashed_password = Column(String)

    # Columna 'role': tipo texto, por defecto se asigna "user" (puede ser "admin" u otros roles si se define)
    role = Column(String, default="user")

    # Columna 'is_active': tipo booleano, indica si el usuario está activo o no, por defecto es True
    is_active = Column(Boolean, default=True)


app/main.py

In [ ]:
# Importa la clase principal de FastAPI, que permite crear la aplicación web
from fastapi import FastAPI

# Importa las rutas relacionadas con autenticación (registro, login, etc.)
from routes import auth_routes

# Importa la función para inicializar el sistema de logs
from core.logger import init_logger

# Importa la clase Base y el motor de conexión a la base de datos
from database.session import Base, engine

# Importa el módulo de modelos (esto asegura que SQLAlchemy reconozca las clases mapeadas)
from models import user_task

# Crea una instancia de la aplicación FastAPI, con un título personalizado
app = FastAPI(title="Sistema de Gestión de Cuentas")

# Crea todas las tablas definidas en los modelos si no existen aún en la base de datos
Base.metadata.create_all(bind=engine)

# Inicializa el logger: se configura el archivo de log y el formato de mensajes
init_logger()

# Incluye las rutas definidas en el módulo auth_routes
# Esto habilita los endpoints como /auth/register, /auth/login, etc.
app.include_router(auth_routes.router)


schemas/user_task.py

In [ ]:
# Importa la clase base para crear modelos Pydantic, y Field para aplicar validaciones adicionales
from pydantic import BaseModel, Field

# Importa Annotated, que permite agregar validaciones al tipo de dato
from typing import Annotated

# Modelo que se usa al registrar un nuevo usuario en el sistema
class UserCreate(BaseModel):
    # Campo 'username': debe ser una cadena entre 4 y 20 caracteres
    username: Annotated[str, Field(min_length=4, max_length=20)]
    
    # Campo 'password': debe tener al menos 6 caracteres
    password: Annotated[str, Field(min_length=6)]

    # Campo 'role': permite indicar el rol del usuario (por ejemplo: "user", "admin")
    role: str

# Modelo que se usa para retornar información del usuario sin incluir datos sensibles
class UserOut(BaseModel):
    # ID del usuario en la base de datos
    id: int

    # Nombre de usuario
    username: str

    # Rol del usuario
    role: str

    # Configuración para permitir la conversión desde objetos ORM como los de SQLAlchemy
    class Config:
        from_attributes = True  # Permite crear este modelo directamente desde objetos de la base de datos

# Modelo que se utiliza para enviar el token JWT como respuesta al iniciar sesión
class Token(BaseModel):
    # Token JWT generado
    access_token: str

    # Tipo de token (usualmente "bearer")
    token_type: str

# Modelo que se utiliza para recibir los datos del formulario de inicio de sesión
class LoginForm(BaseModel):
    # Nombre de usuario para iniciar sesión
    username: str

    # Contraseña del usuario
    password: str


services/auth_service.py

In [ ]:
# Importa el contexto de encriptación de contraseñas con bcrypt
from passlib.context import CryptContext

# Importa funciones y excepciones para codificar y decodificar JWT
from jose import jwt, JWTError

# Importa funciones de fecha y hora para generar expiraciones de tokens
from datetime import datetime, timedelta

# Importa el tipo de sesión de SQLAlchemy para interactuar con la base de datos
from sqlalchemy.orm import Session

# Importa la excepción HTTPException para lanzar errores personalizados en FastAPI
from fastapi import HTTPException, Depends

# Importa el esquema de seguridad para autenticación con token (OAuth2 con password bearer)
from fastapi.security import OAuth2PasswordBearer

# Importa el modelo ORM del usuario (tabla `users`)
from models.user_task import User

# Importa el esquema de entrada para crear usuarios (username, password, role)
from schemas.user_task import UserCreate

# Importa las variables de configuración definidas en el archivo `.env`
from core.config import SECRET_KEY, ALGORITHM, ACCESS_TOKEN_EXPIRE_MINUTES

# Importa la función para obtener la sesión de base de datos como dependencia
from database.session import get_db

# Crea un contexto de encriptación con el algoritmo bcrypt (seguro y recomendado)
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# Define el esquema de autenticación OAuth2. Se espera que los tokens se obtengan en /auth/login
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="/auth/login")

# Función para registrar un nuevo usuario en la base de datos
def create_user(user: UserCreate, db: Session):
    # Encripta la contraseña usando bcrypt
    hashed_pw = pwd_context.hash(user.password)

    # Crea una instancia del modelo User con los datos recibidos
    db_user = User(username=user.username, hashed_password=hashed_pw, role=user.role)

    # Agrega el nuevo usuario a la sesión de la base de datos
    db.add(db_user)

    # Guarda los cambios en la base de datos
    db.commit()

    # Refresca el objeto con los datos actuales de la base de datos (incluye ID asignado)
    db.refresh(db_user)

    # Retorna el usuario creado
    return db_user

# Función para autenticar a un usuario durante el login
def authenticate_user(username: str, password: str, db: Session):
    # Busca al usuario por nombre de usuario
    user = db.query(User).filter_by(username=username).first()

    # Si no existe el usuario o la contraseña no coincide, retorna None
    if not user or not pwd_context.verify(password, user.hashed_password):
        return None

    # Si las credenciales son válidas, retorna el objeto del usuario
    return user

# Función para generar un token JWT válido
def create_access_token(data: dict, db: Session):
    # Clona el diccionario con los datos del usuario (normalmente contiene el username)
    to_encode = data.copy()

    # Calcula la fecha de expiración sumando minutos definidos en la configuración
    expire = datetime.now() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)

    # Agrega la expiración al payload
    to_encode.update({"exp": expire})

    # Codifica el JWT con la clave secreta y el algoritmo definido
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

# Función para obtener el usuario actual autenticado, usando el token enviado
def get_current_user(token: str = Depends(oauth2_scheme), db: Session = Depends(get_db)):
    try:
        # Decodifica el token para obtener los datos del usuario (payload)
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])

        # Busca el usuario en la base de datos usando el campo "sub" del token (sujeto)
        user = db.query(User).filter_by(username=payload["sub"]).first()

        # Si el usuario no existe, lanza un error 403 (Prohibido)
        if user is None:
            raise HTTPException(status_code=403, detail="Usuario no encontrado")

        # Si todo es correcto, retorna el usuario autenticado
        return user

    # Captura errores de decodificación del token (token inválido o expirado)
    except JWTError:
        raise HTTPException(status_code=403, detail="Token inválido")


routes/auth_routes.py

In [ ]:
# Importa herramientas de FastAPI para crear rutas, manejar dependencias y lanzar excepciones HTTP
from fastapi import APIRouter, Depends, HTTPException

# Importa el formulario estandarizado OAuth2 para recibir username y password
from fastapi.security import OAuth2PasswordRequestForm

# Importa la clase Session de SQLAlchemy para manejar la conexión con la base de datos
from sqlalchemy.orm import Session

# Importa la función para obtener una sesión activa de la base de datos (inyección de dependencia)
from database.session import get_db

# Importa las funciones del servicio de autenticación (registro, login, token, usuario actual)
from services.auth_service import create_user, authenticate_user, create_access_token, get_current_user

# Importa los esquemas para entrada y salida de datos del usuario y del token
from schemas.user_task import UserCreate, UserOut, Token

# Importa el modelo ORM del usuario para usarlo en validaciones y dependencias
from models.user_task import User

# Crea un router para todas las rutas relacionadas con autenticación
router = APIRouter(prefix="/auth", tags=["auth"])

# Ruta para registrar un nuevo usuario
@router.post("/register", response_model=UserOut)
def register(user: UserCreate, db: Session = Depends(get_db)):
    # Validación personalizada: username debe tener entre 4 y 20 caracteres
    if len(user.username) < 4 or len(user.username) > 20:
        raise HTTPException(status_code=400, detail="El nombre de usuario debe tener entre 4 y 20 caracteres")
    
    # Validación personalizada: la contraseña debe tener al menos 6 caracteres
    if len(user.password) < 6:
        raise HTTPException(status_code=400, detail="La contraseña debe tener al menos 6 caracteres")
    
    # Validación personalizada: solo se permite rol "user" o "admin"
    if user.role not in ["user", "admin"]:
        raise HTTPException(status_code=400, detail="Rol inválido")
    
    # Si todas las validaciones pasan, se crea el usuario en la base de datos
    return create_user(user, db)

# Ruta para hacer login y obtener el token JWT
@router.post("/login", response_model=Token)
def login(form_data: OAuth2PasswordRequestForm = Depends(), db: Session = Depends(get_db)):
    # Verifica que las credenciales sean válidas
    user = authenticate_user(form_data.username, form_data.password, db)

    # Si no se autentica, lanza error 401 (no autorizado)
    if not user:
        raise HTTPException(status_code=401, detail="Credenciales inválidas")
    
    # Genera el token JWT con username y rol como payload
    token = create_access_token({"sub": user.username, "role": user.role}, db)
    
    # Retorna el token y su tipo (bearer)
    return {"access_token": token, "token_type": "bearer"}

# Ruta para obtener el usuario actualmente autenticado (token obligatorio)
@router.get("/me", response_model=UserOut)
def get_me(current_user: User = Depends(get_current_user)):
    # Retorna los datos del usuario actual si el token es válido
    return current_user

# Ruta protegida que solo puede ser accedida por usuarios con rol "admin"
@router.get("/admin")
def admin_route(current_user: User = Depends(get_current_user)):
    # Si el usuario no es administrador, lanza un error 403 (prohibido)
    if current_user.role != "admin":
        raise HTTPException(status_code=403, detail="Solo administradores")
    
    # Si es administrador, retorna un mensaje de bienvenida
    return {"msg": f"Bienvenido, admin {current_user.username}"}


streamlit_app.py

In [ ]:
# Importa la biblioteca Streamlit para crear interfaces gráficas web
import streamlit as st

# Importa requests para hacer llamadas HTTP a la API FastAPI
import requests

# Dirección base de la API local de FastAPI
API_BASE_URL = "http://localhost:8000"

# Configura el título y el ícono de la pestaña del navegador
st.set_page_config(page_title="Gestión de Cuentas", page_icon="🔐")

# Título principal de la página
st.title("Sistema de Gestión de Cuentas")

# Menú lateral para seleccionar entre Registro y Login
menu = ["Registro", "Login"]
choice = st.sidebar.selectbox("Menú", menu)

# Variables de sesión para manejar login persistente
if "token" not in st.session_state:
    st.session_state.token = None
if "user" not in st.session_state:
    st.session_state.user = None

# --- REGISTRO DE USUARIOS ---
if choice == "Registro":
    st.subheader("Crear nuevo usuario")
    
    # Formulario para registrar nuevos usuarios
    with st.form("register_form", clear_on_submit=True):
        username = st.text_input("Usuario", help="4-20 caracteres")
        password = st.text_input("Contraseña", type="password", help="Mínimo 6 caracteres")
        role = st.selectbox("Rol", ["user", "admin"])
        submit = st.form_submit_button("Registrar")
        
        if submit:
            # Validaciones mínimas antes de enviar al backend
            if len(username) < 4 or len(username) > 20:
                st.error("El usuario debe tener entre 4 y 20 caracteres")
            elif len(password) < 6:
                st.error("La contraseña debe tener al menos 6 caracteres")
            else:
                # Datos a enviar a la API para crear usuario
                data = {"username": username, "password": password, "role": role}
                try:
                    # Enviar POST a /auth/register
                    r = requests.post(f"{API_BASE_URL}/auth/register", json=data)
                    if r.status_code == 200:
                        st.success(f"✅ Usuario '{username}' creado correctamente")
                    else:
                        st.error(f"❌ Error: {r.json().get('detail', r.text)}")
                except Exception as e:
                    st.error(f"❌ Error de conexión: {e}")

# --- LOGIN DE USUARIOS ---
if choice == "Login":
    st.subheader("Iniciar sesión")
    
    # Formulario para ingresar usuario y contraseña
    with st.form("login_form", clear_on_submit=True):
        username = st.text_input("Usuario")
        password = st.text_input("Contraseña", type="password")
        submit = st.form_submit_button("Entrar")
        
        if submit:
            # Validación básica
            if not username or not password:
                st.warning("⚠️ Completa usuario y contraseña")
            else:
                # Datos a enviar al endpoint /auth/login
                data = {"username": username, "password": password}
                try:
                    # Login usando x-www-form-urlencoded (OAuth2PasswordRequestForm)
                    r = requests.post(f"{API_BASE_URL}/auth/login", data=data)
                    if r.status_code == 200:
                        # Guarda el token JWT en sesión
                        token = r.json()["access_token"]
                        st.session_state.token = token
                        st.success("✅ Login exitoso")
                        
                        # Llama a /auth/me para obtener los datos del usuario
                        headers = {"Authorization": f"Bearer {token}"}
                        r2 = requests.get(f"{API_BASE_URL}/auth/me", headers=headers)
                        if r2.status_code == 200:
                            st.session_state.user = r2.json()
                        else:
                            st.session_state.user = None
                    else:
                        st.error("❌ Credenciales inválidas")
                except Exception as e:
                    st.error(f"❌ Error de conexión: {e}")

    # Mostrar información del usuario si está autenticado
    if st.session_state.token and st.session_state.user:
        st.success(f"👋 Bienvenido, {st.session_state.user['username']} (rol: {st.session_state.user['role']})")
        
        # Botón para cerrar sesión
        if st.button("🚪 Cerrar Sesión"):
            st.session_state.token = None
            st.session_state.user = None
            st.success("✅ Sesión cerrada correctamente")
            st.rerun()
        
        # Si el usuario es admin, intenta acceder a ruta protegida para administradores
        if st.session_state.user["role"] == "admin":
            headers = {"Authorization": f"Bearer {st.session_state.token}"}
            r = requests.get(f"{API_BASE_URL}/auth/admin", headers=headers)
            if r.status_code == 200:
                st.info("🔐 Tienes acceso a la ruta de administrador.")
            else:
                st.warning("⚠️ No tienes acceso a la ruta de administrador.")
        else:
            st.info("👤 Eres un usuario estándar.")


.env

In [ ]:
SECRET_KEY='123456'
POSTGRES_USER=postgres
POSTGRES_PASSWORD=1126254560
POSTGRES_DB=secure_db
POSTGRES_HOST=localhost
POSTGRES_PORT=5432
